# Truncated Series Method for MVNQF
Implementation of the method to compute the CDF of a MVNQF using a truncated series representation described in: 
https://reader.elsevier.com/reader/sd/pii/S0167947308005653?token=2CBECC11CD4A4A6EB481FFB9A1CC3226413187AC4274D47A434BBD09279A6CF441F232D25D7E035A2566AC4468729B82

The sum of the coefficients give us a bound on error, so the purpose of this notebook is to compute the coefficients and examine how many are needed to get a useful bound on error.

In [67]:
import numpy as np
import scipy.linalg as sla
from scipy import random
import math

In [159]:
def random_psd_matrix(n):
    """
    Generate a random n dimensional PSD matrix.
    """
    random_matrix = random.rand(n, n)
    psd_matrix = np.dot(random_matrix, random_matrix.transpose())
    return psd_matrix

def compute_lambdas_deltas(mux, Sigma, Q, diagonalization_error_tolerance = 1e-10):
    """
    
    """
    
    sqrt_Sigma = sla.fractional_matrix_power(Sigma, 0.5)
    B = sqrt_Sigma @ Q @ sqrt_Sigma
    eigvals, P = np.linalg.eig(B) # P is the matrix of eigenvectors of B
    
    # Lambda should be a diagonal matrix (non-diagonal entries may be non-zero but tiny due to floating point precision)
    Lambda = P.T @ B @ P
    lambdas = np.diag(Lambda)
    
    # Compute deltas
    inv_sqrt_Sigma = np.linalg.inv(sqrt_Sigma)
    muy = P @ inv_sqrt_Sigma @ mux
    deltas = np.square(muy.flatten()) # Elementwise square of muy
    
    # Test the Lambda is approximately a diagonal matrix
    non_diag_entries = Lambda[np.where(~np.eye(Lambda.shape[0],dtype=bool))]
    assert max(abs(non_diag_entries)) < diagonalization_error_tolerance
    return lambdas, deltas

def compute_gk(gammas, gamma_ks, gammas_kminus1, deltas, k):
    """
    Compute g as defined in the paper
    """
    m = len(gamma_ks)
    # Compute values that are dependent on k
    term2 = k * sum([deltas[i] * gammas_kminus1[i] * (1 - gammas[i]) for i in range(m)])
    gk = 0.5 * (sum(gamma_ks) + term2)
    return gk

def power_matrix(one_d_array, kmax):
    """
    Given a one_d_array, construct a matrix s.t. the ith index of the matrix corresponds to
    one_d_array to the power of i. The matrix will have kmax + 1 number of rows
    Args:
        one_d_array
    """
    one_d_array_powers = np.zeros((kmax + 1, m))
    for i in range(kmax + 1):
        one_d_array_powers[i] = np.power(one_d_array, i)
    return one_d_array_powers
    
def update_ck(cks, gks, kmax):
    """
    Args:
        cks (list of floats): list of coefficients s.t. the ith index corresponds to c_i
        gks (list of floats): constant length list s.t. the ith index corresponds to g_i
    """
    k = len(cks)
    if k >= kmax:
        return
    ck = (1.0/k) * sum([gks[k - r] * cks[r] for r in range(k)])
    cks.append(ck)
    update_ck(cks, gks, kmax)
    return

def compute_series_coefficients(mux, Sigma, Q, kmax):
    # Compute values that are independent of k
    m = np.linalg.matrix_rank(Q)
    lambdas, deltas = compute_lambdas_deltas(mux, Sigma, Q)
    beta = 0.99 * min(lambdas) # Beta can be any value between 0 and the minimum of the lambdas
    h_tilde = m # In the paper, h_tilde is the sum of h_i for i = 1,...,m but h_i = 1?
    gammas = [1 -  beta/lam for lam in lambdas]
    d = np.prod([(beta/lam)**0.5 for lam in lambdas])
    e = sum(deltas)

    # The ith row (by index) of this matrix contains gammas raised to the ith
    gammas_powers = power_matrix(gammas, kmax)
    gks = [compute_gk(gammas_powers[1], gammas_powers[k], gammas_powers[k - 1], deltas, k) for k in range(kmax + 1)]
    c0 = d * math.exp(-0.5 * e)
    cks = [c0] # c0 has a different formula than ck for k > 0
    update_ck(cks, gks, kmax)
    return cks

In [167]:
Q = random_psd_matrix(3)
Sigma = random_psd_matrix(3)
mux = np.ones((3, 1))
kmax = 1500

In [168]:
cks = compute_series_coefficients(mux, Sigma, Q, kmax)
print(sum(cks))

0.24510643732632473
